In [1]:
import numpy as np
import ccxt
from ccxt.base.exchange import Exchange
from ccxt.base.errors import InsufficientFunds
from ccxt.base.errors import BadRequest
import schedule
import time
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import ssl
import json
import requests
from datetime import date
from songline import Sendline

In [2]:
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

In [3]:
url = 'xx'
token = 'x'
headers = {'content-type':'application/x-www-form-urlencoded','Authorization':'Bearer '+token}
messenger = Sendline(token)

In [4]:
api_key = 'xxx' # API Keys
secret = 'xx' # API Secret
subaccount = 'x'  # Choose sub-acc

ftx = ccxt.ftx({
    'api_key': api_key,
    'secret': secret,
    'enableRateLimit': True, 'rateLimit': 2000
})

ftx.headers = {'FTX-SUBACCOUNT': subaccount,}

In [6]:
scope = ['https://www.googleapis.com/auth/spreadsheets']
credentials = ServiceAccountCredentials.from_json_keyfile_name('zz.json', scope)
gc = gspread.authorize(credentials)
sheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/z")
worksheet = sheet.get_worksheet(0) # sheet index in spreadsheets

In [5]:
pair = 'BULL/USD'
quote_currency = 'USD'
symbol = 'BULL'
thr = 1.0 # %
cash = [x['usdValue'] for x in (ftx.fetch_balance()['info']['result']) if x['coin'] == 'USD'][0]
nav = [x['usdValue'] for x in (ftx.fetch_balance()['info']['result']) if x['coin'] == symbol][0]

In [6]:
# last_price = ftx.fetch_ticker(pair)['last']
# ftx.create_market_buy_order(pair, nav/last_price)

In [7]:
# def cancel_orders():
#     ftx.cancel_all_orders()                                                                

In [8]:
def cancel_orders():
    ftx.cancel_order(ftx.fetch_open_orders()[-1]['id'])

In [10]:
def open_orders():
    balance = ftx.fetch_balance() 
    amount = balance[symbol]['total']
    
    #Price
    price_up = (nav/amount) * (1+(thr/100))    # = +1.5% threshold reb
    price_dw = (nav/amount) * (1-(thr/100))   # = -1.5% threshold reb 
#     print(price_up,price_dw)
    
    nav_up = price_up * amount
    nav_dw = price_dw * amount
    
    #Cal amount
    dif_up = abs(nav-nav_up) #2/200
    dif_dw = abs(nav-nav_dw)
    
    amount_up = dif_up/price_up
    amount_dw = dif_dw/price_dw
#     print(amount_up,amount_dw)
    
    #Place Order
    sell_order = ftx.create_order(pair,'limit','sell', amount_up, price_up)
    buy_order = ftx.create_order(pair,'limit','buy', amount_dw, price_dw)
    acc()                                                               

In [11]:
def acc():
    d = ftx.fetchMyTrades(pair)
    balance = ftx.fetch_balance()
    
    datetime = d[-1]['datetime']
    side = d[-1]['side']
    price = d[-1]['price']
    cash_val = [x['usdValue'] for x in (balance['info']['result']) if x['coin'] == 'USD'][0]
    nav_val = [x['usdValue'] for x in (balance['info']['result']) if x['coin'] == symbol][0]
    port_val = round(sum(map(lambda x: x['usdValue'], balance['info']['result'])),4)
    amount_hold = balance['total'][symbol]
    
    worksheet.insert_row([datetime, side, price, cash_val, nav_val, port_val, amount_hold], 2)

In [12]:
def check_orders():
    try:
        my_open_orders = ftx.fetch_open_orders(pair)
        if len(my_open_orders) == 1:   
            cancel_orders()
            open_orders()

        elif len(my_open_orders) == 2:   
            pass

        elif len(my_open_orders) == 0:   
            open_orders()
            
        else:
            cancel_orders()
            
    except InsufficientFunds as e:
        global go
        go = False
        time.sleep(1)
    except ccxt.NetworkError as e:
        print(ftx.id, 'fetch_order_book failed due to a network error:', str(e))
        time.sleep(1)
        check_orders()
    except ccxt.ExchangeError as e:
        print(ftx.id, 'fetch_order_book failed due to exchange error:', str(e))
        time.sleep(1)
        check_orders()
    except Exception as e:
        print(ftx.id, 'fetch_order_book failed with:', str(e))
        time.sleep(1)
        check_orders()

In [13]:
def check_compound():
    try:
        global cash, nav

        msg = 'check if compound: nav={}, cash={}'.format(nav, cash)
        r = requests.post(url, headers=headers, data = {'message':msg})
        curr_cash = [x['usdValue'] for x in (ftx.fetch_balance()['info']['result']) if x['coin'] == 'USD'][0]
        
        if curr_cash > nav:

            diff = (curr_cash - nav)/2
            nav = nav + diff
            cash = curr_cash - diff
            
            msgg = 'do compound: new nav={}, new cash={}'.format(nav, cash)
            messenger.sendtext(msgg)
            
            ftx.cancel_all_orders()
            open_orders()

            
    except InsufficientFunds as e:
        global go
        go = False
        time.sleep(1)
    except ccxt.NetworkError as e:
        print(ftx.id, 'fetch_order_book failed due to a network error:', str(e))
        time.sleep(1)
        check_orders()
    except ccxt.ExchangeError as e:
        print(ftx.id, 'fetch_order_book failed due to exchange error:', str(e))
        time.sleep(1)
        check_orders()
    except Exception as e:
        print(ftx.id, 'fetch_order_book failed with:', str(e))
        time.sleep(1)
        check_orders()

In [1]:
schedule.every(2).seconds.do(check_orders)
schedule.every(0.08).days.do(check_compound)
check_compound()
go = True
while go:
    schedule.run_pending()
    time.sleep(1)